In [3]:
from sklearn.datasets import make_regression

# Generate regression toy data
n_samples = 1000
n_features = 5

X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42)

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)

X_train = X[:train_size]
y_train = y[:train_size]

# Test data
X_test = X[train_size:]
y_test = y[train_size:]

In [4]:
from sklearn.model_selection import cross_val_score


def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [10]:
dt_bo = BayesianOptimization(f=lambda n_estimators, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'n_estimators': int(round(n_estimators)),
                                        'max_depth': int(round(max_depth)),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': int(round(max_leaf_nodes))
                                    }, X_train, y_train),
                             pbounds=params_ranges)
results = dt_bo.maximize(n_iter=5, init_points=20)

|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 46.61     | 4.134     | 0.46      | 22.08     | 9.013     | 0.2209    | 41.34     |
| 2         | 36.43     | 12.73     | 0.1274    | 62.41     | 1.791     | 0.04717   | 29.28     |
| 3         | 46.17     | 15.91     | 0.3348    | 43.93     | 8.521     | 0.1384    | 30.4      |
| 4         | 48.93     | 6.856     | 0.3888    | 32.59     | 9.715     | 0.1876    | 41.59     |
| 5         | 46.0      | 1.771     | 0.453     | 85.14     | 4.254     | 0.1906    | 39.71     |
| 6         | 28.97     | 19.53     | 0.8609    | 26.85     | 3.368     | 0.03503   | 42.77     |
| 7         | 51.6      | 1.258     | 0.7595    | 29.45     | 6.586     | 0.3973    | 20.35     |
| 8         | 53.54     | 13.96     | 0.5825    | 83.65     | 2.978     | 0.422     | 30.61     |
| 9         | 42.81 

In [5]:
all_models = {}

# Random Forest

**Feature selection**

In [15]:
from sklearn.ensemble import RandomForestRegressor

# Assuming you have X_train and y_train defined for training data

# Initialize the Random Forest model
rf_model = RandomForestRegressor()

# Fit the Random Forest model to the training data
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_

# Create a list of feature names or indices paired with their importances
feature_importances = list(zip(range(X_train.shape[1]), importances))

# Sort the features based on importance in descending order
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Print the ranked feature importances
print("Feature Importances:")
for feature_index, importance in feature_importances:
    print(f"Feature {feature_index}: {importance}")


Feature Importances:
Feature 2: 0.5484287743797435
Feature 3: 0.17394567943882905
Feature 4: 0.13497297114048568
Feature 1: 0.08208743352080566
Feature 0: 0.06056514152013607


**Hyperparameter optimalisatie**

In [13]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization

params_ranges = {
    'n_estimators': (10, 100),
    'max_depth': (1, 20),
    'min_samples_leaf': (1, 10),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_features': (0.1, 1),
    'max_leaf_nodes': (10, 100)
}

# Example usage with Random Forest
model = RandomForestRegressor
dt_bo = BayesianOptimization(f=lambda n_estimators, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'n_estimators': int(round(n_estimators)),
                                        'max_depth': int(round(max_depth)),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': int(round(max_leaf_nodes))
                                    }, X_train, y_train),
                             pbounds=params_ranges)
results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model = model(
    n_estimators=int(round(params['n_estimators'])),
    max_depth=int(round(params['max_depth'])),
    min_samples_leaf=round(params['min_samples_leaf']),
    min_weight_fraction_leaf=params['min_weight_fraction_leaf'],
    max_features=params['max_features'],
    max_leaf_nodes=int(round(params['max_leaf_nodes']))
)

# Fit the model
best_model.fit(X_train, y_train)

|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 56.57     | 6.18      | 0.1356    | 96.53     | 1.01      | 0.417     | 20.46     |
| 2         | 44.13     | 7.106     | 0.7864    | 77.59     | 7.03      | 0.2144    | 52.05     |
| 3         | 49.95     | 14.63     | 0.7363    | 33.25     | 3.595     | 0.3243    | 10.47     |
| 4         | 53.18     | 11.58     | 0.4948    | 30.3      | 1.18      | 0.4085    | 45.14     |
| 5         | 56.2      | 17.34     | 0.2173    | 58.35     | 9.396     | 0.4473    | 95.6      |
| 6         | 50.6      | 15.72     | 0.9867    | 71.45     | 3.314     | 0.4187    | 96.27     |
| 7         | 45.6      | 17.99     | 0.875     | 73.23     | 4.524     | 0.237     | 68.85     |
| 8         | 40.02     | 19.35     | 0.4018    | 66.27     | 2.867     | 0.1252    | 24.15     |
| 9         | 49.05 

RandomForestRegressor(max_depth=18, max_features=0.31942131115922046,
                      max_leaf_nodes=69, min_samples_leaf=8,
                      min_weight_fraction_leaf=0.49934071613135156,
                      n_estimators=85)

In [7]:
# Append model to list
all_models['Random forest'] = best_model